In [1]:
from CleanText import SentTokenize
from  RemoveStopWords import stopWords_removal
from PorterStemmer import porter_stemmer
import csv, json
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import pickle


2024-11-16 02:25:48.249295: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 02:25:48.266324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731704148.286555  375563 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731704148.292667  375563 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 02:25:48.313813: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
def pipeline(input_phrase):
    obj = SentTokenize(phrase=input_phrase) 
    obj = obj.get_tokenized_sentence() 
    print(f"Tokenized sentence: {obj}")

    obj = stopWords_removal(tokenized_sentence=obj) 
    obj = obj.get_refined_tokeinzed_sentence()
    print(f"After stop words removal: {obj}")

    obj = porter_stemmer(tokenised_phrase=obj)
    obj = obj.get_stemmed_tokens() 
    print(f"After stemming with porters algorithm: {obj}") 


    return obj

In [8]:

with open('updated_corpus/updated_corpus.json', 'r') as f:
    data = json.load(f) 
    f.close()

        

def preprocess_data(data):
    rows = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            processed_pattern = pipeline(pattern)
            row = {
                "input_text": " ".join(processed_pattern),
                "target": intent["tag"]
            }
            rows.append(row)
    return rows

# Transform the data to CSV format
preprocessed_data = preprocess_data(data)

with open("updated_preprocessed_data.csv", "w", newline="") as csvfile:
    fieldnames = ["input_text", "target"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in preprocessed_data:
        writer.writerow(row)



Tokenized sentence: ['why', 'can', 't', 'i', 'connect', 'to', 'wifi']
After stop words removal: ['connect', 'wifi']
After stemming with porters algorithm: ['connect', 'wifi']
Tokenized sentence: ['how', 'do', 'i', 'fix', 'wifi', 'connection', 'issues']
After stop words removal: ['fix', 'wifi', 'connection', 'issues']
After stemming with porters algorithm: ['fix', 'wifi', 'connect', 'issu']
Tokenized sentence: ['my', 'wifi', 'is', 'not', 'working']
After stop words removal: ['wifi', 'working']
work
After stemming with porters algorithm: ['wifi', 'wor']
Tokenized sentence: ['why', 'is', 'my', 'wifi', 'connection', 'unstable']
After stop words removal: ['wifi', 'connection', 'unstable']
After stemming with porters algorithm: ['wifi', 'connect', 'unstab']
Tokenized sentence: ['how', 'do', 'i', 'reconnect', 'to', 'wifi', 'after', 'losing', 'connection']
After stop words removal: ['reconnect', 'wifi', 'losing', 'connection']
los
After stemming with porters algorithm: ['reconnect', 'wifi', 'l

In [33]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import Callback

# Load and preprocess data
data = pd.read_csv('updated_preprocessed_data.csv')


# Prepare data
X = data['input_text']
y = data['target']

# Create Tokenizer and fit on text data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(X)

# Convert the target classes into a list and map them
classes_set = list(set(y))
class_to_index = {cls: idx for idx, cls in enumerate(classes_set)}

# Convert targets to integer labels
y_encoded = y.map(class_to_index)

# Prepare input sequences
max_len = 20  # Max sequence length
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, np.array(y_encoded), test_size=0.2, random_state=42)

# Check the shapes of X and y
print(X_train.shape)  # Should be (num_samples, max_len)
print(y_train.shape)  # Should be (num_samples,)

# Create and compile the model
model = Sequential()
model.add(Embedding(2000, 10, input_length=max_len))
np.random.seed(7)
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(32))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(classes_set), activation='softmax'))  # Number of unique classes

# Compile the model with accuracy and precision
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


# Train the model
model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_val, y_val))

# Final evaluation
# Final evaluation
results = model.evaluate(X_val, y_val, verbose=0)
loss = results[0]  # The first value is the loss
accuracy = results[1]  # The second value is accuracy


# Print evaluation results
print(f"Final Validation Loss: {loss}, Final Validation Accuracy: {accuracy}")

# Save the model
model.save('Model_Results_test/my_model.keras')  # Saves the entire model in the .keras format

# Save the tokenizer using pickle
with open('Model_Results_test/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Save the classes mapping using pickle
with open('Model_Results_test/classes_set.pkl', 'wb') as f:
    pickle.dump(classes_set, f)



print("Model, Tokenizer, and Classes Mapping Saved!")




(288, 20)
(288,)
Epoch 1/300


/home/anudeep/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0497 - loss: 3.5846 - val_accuracy: 0.0000e+00 - val_loss: 3.5850
Epoch 2/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0471 - loss: 3.5839 - val_accuracy: 0.0000e+00 - val_loss: 3.5899
Epoch 3/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0408 - loss: 3.5801 - val_accuracy: 0.0000e+00 - val_loss: 3.5963
Epoch 4/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0261 - loss: 3.5808 - val_accuracy: 0.0000e+00 - val_loss: 3.6000
Epoch 5/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0308 - loss: 3.5791 - val_accuracy: 0.0000e+00 - val_loss: 3.6051
Epoch 6/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0458 - loss: 3.5776 - val_accuracy: 0.0000e+00 - val_loss: 3.6165
Epoch 7/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0145 - loss: 3.5822 - val_accuracy: 0.0000e+00 - val_loss: 3.6176
Epoch 8/300
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0485 - loss: 3.5728 - val_accuracy: 0.

In [ ]:
## testing the things

# Now you can later load the model and tokenizer like this:

# 1. Load the saved model
from tensorflow.keras.models import load_model
from CleanText import Token_to_Sent 
import pickle


loaded_model = load_model('Model_Results_test/my_model.keras')

max_len = 20

# 2. Load the saved tokenizer
with open('Model_Results_test/tokenizer.pkl', 'rb') as f:
    loaded_tokenizer = pickle.load(f)

# 3. Load the classes mapping
with open('Model_Results_test/classes_set.pkl', 'rb') as f:
    loaded_classes_set = pickle.load(f)

print("Model, Tokenizer, and Classes Mapping Loaded!")

# Function to optimize and predict new input
def optimize_phrase(sentence):
    sequences = loaded_tokenizer.texts_to_sequences([sentence])
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences

# Example prediction


2024-11-16 02:26:26.517988: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-11-16 02:26:26.518038: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: anudeep-OMEN-by-HP-Gaming-Laptop-16-wf1xxx
2024-11-16 02:26:26.518050: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: anudeep-OMEN-by-HP-Gaming-Laptop-16-wf1xxx
2024-11-16 02:26:26.518222: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 560.35.3
2024-11-16 02:26:26.518247: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 560.35.3
2024-11-16 02:26:26.518253: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 560.35.3


Model, Tokenizer, and Classes Mapping Loaded!


In [5]:

def get_predicted_class(sentence):
    refined_sentence = pipeline(input_phrase=sentence)
    ts = Token_to_Sent() 
    optimized_input = optimize_phrase(ts.token_to_sent(tokenized_sentence=refined_sentence))
    prediction = loaded_model.predict(optimized_input)
    predicted_class = loaded_classes_set[np.argmax(prediction)]

    print(f"Predicted Class: {predicted_class}") 
    return predicted_class

### New model

### Testing the value

In [11]:
sentence = "My ram got freeze"  
l = get_predicted_class(sentence) 
print(l)

Tokenized sentence: ['my', 'ram', 'got', 'freeze']
After stop words removal: ['ram', 'got', 'freeze']
After stemming with porters algorithm: ['ram', 'got', 'freez']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Class: RAM_frequency_issue
RAM_frequency_issue
